In [1]:
# --- Imports ---
import os
import random
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report, accuracy_score
import re

In [2]:
# --- Configuration ---
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"

CSV_PATH = r"C:\Users\stdFurqan\Desktop\pastho_rob\dataset_test_30 - Sheet1.csv"
TEXT_COL = "Text"
LABEL_COL = "label"
LABELS = ["Positive", "Negative", "Neutral"]
SEED = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Reproducibility ---
def set_seed(seed=20):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(SEED)

In [3]:
# ✅ Local path where model is stored
MODEL_PATH = r"C:\Users\stdFurqan\Downloads\lama_models_download\LAMA_3.2(1b)"

print(f"🧠 Loading model from {MODEL_PATH} ...")

# ✅ Load tokenizer from local folder
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# ✅ Load model from local folder
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,   # ✅ best for 40-series
    device_map="auto"
)

model.config.pad_token_id = tokenizer.pad_token_id
model.eval()

print("✅ Model loaded successfully from local directory!")

🧠 Loading model from C:\Users\stdFurqan\Downloads\lama_models_download\LAMA_3.2(1b) ...
✅ Model loaded successfully from local directory!


In [4]:
# print("You are an intelligent Pashto language assistant. "
#     "Classify the sentiment of the given text as Positive, Negative, or Neutral based on the examples below.\n\n"
    
#     "=== Positive Examples ===\n"
#     "1 Text: د يک‌شنبې له نښتې وروسته د سپين بولدک-چمن لارې تړل کېدل\n   Label: Positive\n"
#     "2 Text: سرچينې ليکلي: «هېڅوک بايد د خپلو بنسټيزو حقونو او د نورو د حقونو په دفاع کې د خبرو کولو له امله ونه نيول شي.»\n   Label: Positive\n"
#     "3 Text: هلمند کې د يوه جمنازيوم جوړولو چارې پيل شوې\n   Label: Positive\n"
#     "4 Text: د ننګرهار د کرنې رياست د کرنې امر نعمت‌الله اکبري پژواک اژانس ته وويل، په دغه ولايت کې د تېر کال په پرتله د ګنيو په حاصلاتو کې شاوخوا ۱۷ زره ټنه ډېروالى راغلي دي.\n   Label: Positive\n\n"

#     "=== Negative Examples ===\n"
#     "5 Text: ننګرهار کې د ترافيکي پېښې له امله څلورو کسانو ته مرګ‌ژوبله اوښتې ده\n   Label: Negative\n"
#     "6 Text: نوموړي وايي، دوي څلور ډوله خلکو ته د پاسپورت ترلاسه کولو کې لومړيتوب ورکوي، محصلين، لوبغاړي، سوداګر او عاجل ناروغان.\n   Label: Negative\n"
#     "7 Text: هغه ويلي وؤ امارت اسلامي افغانستان دبرتانيه سره دپرله پسې تماسونو نه وروسته دغه نيول شوي کسان خوشې کړي اؤ خپل هيواد ته يې سپارلي دي.\n   Label: Negative\n"
#     "8 Text: سرچينې زياته کړې، په دې توګه ملګرو ملتونو خپلو ټولو نارينه او ښځينه کارکوونکو ته لارښوونه وکړه چې تر دويم لارښوود پورې دندو ته ولاړ نه شي.\n   Label: Negative\n\n"

#     "=== Neutral Examples ===\n"
#     "9 Text: سرچينې زياته کړې، چې اوس‌مهال د ملګرو ملتونو لږ شمېر کارکوونکي د اړينو کارونو د ترسراوي په موخه دندې ته حاضرېږي.\n   Label: Neutral\n"
#     "10 Text: اوچا: که بېړنۍ مرستې و نه رسېږي، ميليونونه افغانان به د قحطۍ له ګواښ سره مخ شي\n   Label: Neutral\n"
#     "11 Text: د جاپان په وزيراعظم بريد،ملزم ونيولې شو\n   Label: Neutral\n"
#     "12 Text: اقتصاد پوهان وايي څو پورې چې نړيوالې مالياتي ادارې د پاکستان د پور پروګرام نه وي بحال کړي او دوست هېوادونو ورته پېسې نه وي ورکړي اقتصادي مشکلات به يې سېوا کيږي.\n   Label: Neutral\n\n"
    
#     "Now classify the following text in the same way.\n\n"
#     "Text: {text}\n\n"
#     "Label:")

In [5]:


# --- Load dataset ---
df = pd.read_csv(CSV_PATH)
texts = df[TEXT_COL].astype(str).tolist()
true_labels = df[LABEL_COL].astype(str).tolist()

PROMPT_TEMPLATE = (
    "You are an intelligent Pashto language assistant. "
    "Classify the sentiment of the given text as Positive, Negative, or Neutral based on the examples below.\n\n"
    
    "=== Positive Examples ===\n"
    "1 Text: د يک‌شنبې له نښتې وروسته د سپين بولدک-چمن لارې تړل کېدل\n   Label: Positive\n"
    "2 Text: سرچينې ليکلي: «هېڅوک بايد د خپلو بنسټيزو حقونو او د نورو د حقونو په دفاع کې د خبرو کولو له امله ونه نيول شي.»\n   Label: Positive\n"
    "3 Text: هلمند کې د يوه جمنازيوم جوړولو چارې پيل شوې\n   Label: Positive\n"
    "4 Text: د ننګرهار د کرنې رياست د کرنې امر نعمت‌الله اکبري پژواک اژانس ته وويل، په دغه ولايت کې د تېر کال په پرتله د ګنيو په حاصلاتو کې شاوخوا ۱۷ زره ټنه ډېروالى راغلي دي.\n   Label: Positive\n\n"

    "=== Negative Examples ===\n"
    "5 Text: ننګرهار کې د ترافيکي پېښې له امله څلورو کسانو ته مرګ‌ژوبله اوښتې ده\n   Label: Negative\n"
    "6 Text: نوموړي وايي، دوي څلور ډوله خلکو ته د پاسپورت ترلاسه کولو کې لومړيتوب ورکوي، محصلين، لوبغاړي، سوداګر او عاجل ناروغان.\n   Label: Negative\n"
    "7 Text: هغه ويلي وؤ امارت اسلامي افغانستان دبرتانيه سره دپرله پسې تماسونو نه وروسته دغه نيول شوي کسان خوشې کړي اؤ خپل هيواد ته يې سپارلي دي.\n   Label: Negative\n"
    "8 Text: سرچينې زياته کړې، په دې توګه ملګرو ملتونو خپلو ټولو نارينه او ښځينه کارکوونکو ته لارښوونه وکړه چې تر دويم لارښوود پورې دندو ته ولاړ نه شي.\n   Label: Negative\n\n"

    "=== Neutral Examples ===\n"
    "9 Text: سرچينې زياته کړې، چې اوس‌مهال د ملګرو ملتونو لږ شمېر کارکوونکي د اړينو کارونو د ترسراوي په موخه دندې ته حاضرېږي.\n   Label: Neutral\n"
    "10 Text: اوچا: که بېړنۍ مرستې و نه رسېږي، ميليونونه افغانان به د قحطۍ له ګواښ سره مخ شي\n   Label: Neutral\n"
    "11 Text: د جاپان په وزيراعظم بريد،ملزم ونيولې شو\n   Label: Neutral\n"
    "12 Text: اقتصاد پوهان وايي څو پورې چې نړيوالې مالياتي ادارې د پاکستان د پور پروګرام نه وي بحال کړي او دوست هېوادونو ورته پېسې نه وي ورکړي اقتصادي مشکلات به يې سېوا کيږي.\n   Label: Neutral\n\n"

    "Reply with only one word: Positive, Negative, or Neutral.\n"
    "Now classify the following text in the same way.\n\n"
    "Text: {text}\n\n"
    "Label:"
)


pattern = re.compile(r"\b(Positive|Negative|Neutral)\b", re.IGNORECASE)

def extract_label(output):
    m = pattern.search(output)
    return m.group(1).capitalize() if m else "Unknown"

pred_labels = []
batch_size = 4  # smaller batch to fit in T4 VRAM
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    prompts = [PROMPT_TEMPLATE.format(text=t) for t in batch]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=6,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for prompt, full_out in zip(prompts, decoded):
        gen = full_out[len(prompt):].strip() if full_out.startswith(prompt) else full_out
        pred_labels.append(extract_label(gen))

# --- Evaluation ---
report = classification_report(true_labels, pred_labels, labels=LABELS, digits=4, zero_division=0)
acc = accuracy_score(true_labels, pred_labels)
print("\n📊 Classification Report:\n")
print(report)
print(f"Accuracy: {acc:.4f}")

C:\Users\stdFurqan\anaconda3\envs\py312\Lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



📊 Classification Report:

              precision    recall  f1-score   support

    Positive     0.4124    0.3833    0.3974      1573
    Negative     0.3327    0.2883    0.3089      1221
     Neutral     0.2879    0.3612    0.3204      1077

    accuracy                         0.3472      3871
   macro avg     0.3444    0.3443    0.3422      3871
weighted avg     0.3527    0.3472    0.3481      3871

Accuracy: 0.3472
